In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/traincsv/train.csv


In [2]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from category_encoders import CatBoostEncoder

In [3]:
aux = pd.read_csv("/kaggle/input/traincsv/train.csv");

In [4]:
df = aux.copy()

In [5]:
df.columns

Index(['id', 'titulo', 'descripcion', 'tipodepropiedad', 'direccion', 'ciudad',
       'provincia', 'antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 'idzona', 'lat', 'lng', 'fecha',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'precio'],
      dtype='object')

In [6]:
df.dtypes

id                              int64
titulo                         object
descripcion                    object
tipodepropiedad                object
direccion                      object
ciudad                         object
provincia                      object
antiguedad                    float64
habitaciones                  float64
garages                       float64
banos                         float64
metroscubiertos               float64
metrostotales                 float64
idzona                        float64
lat                           float64
lng                           float64
fecha                          object
gimnasio                      float64
usosmultiples                 float64
piscina                       float64
escuelascercanas              float64
centroscomercialescercanos    float64
precio                        float64
dtype: object

In [7]:
df.garages.fillna(0, inplace = True)
df.gimnasio.fillna(0, inplace = True)
df.usosmultiples.fillna(0, inplace = True)
df.piscina.fillna(0, inplace = True)
df.escuelascercanas.fillna(0, inplace = True)
df.centroscomercialescercanos.fillna(0, inplace = True)

In [8]:
df.drop(columns = 'id', inplace = True)
#Saco el id porque es unico para cada propriedad

In [9]:
#Para la antiguedad relleno con el promedio de las 5 que mas aparecen
aux1 = df.antiguedad.value_counts().head(5).reset_index()
aux1['cantidad'] = aux1.antiguedad
aux1['valores'] = aux1['index']*aux1['cantidad']
df.antiguedad.fillna(aux1.valores.sum()/aux1.cantidad.sum(), inplace = True)
df.antiguedad = df.antiguedad.astype(int)

In [10]:
#Reemplazo las habitaciones por las que mas aparecen
df['habitaciones'] = SimpleImputer(strategy='most_frequent').fit_transform(df[['habitaciones']])

In [11]:
#Reemplazo los banos por 1 que es lo minimo posible
df.banos.fillna(1, inplace=True)

In [12]:
#Para los metros totales relleno con 0 al principio y hago el swap
df.metrostotales.fillna(value='0', inplace = True)
df.metroscubiertos.fillna(value='0', inplace = True)
df.metrostotales = df.metrostotales.astype(int)
df.metroscubiertos = df.metroscubiertos.astype(int)
df['metrostotales'] = df[['metrostotales', 'metroscubiertos']].max(axis=1)
df['metroscubiertos'] = df[['metrostotales', 'metroscubiertos']].min(axis=1)

In [13]:
aux2 = df[['idzona', 'ciudad']].copy()
aux2.dropna(inplace = True)
aux2 = aux2.groupby('ciudad').agg({'idzona':'mean'})
ids = aux2.T.to_dict('records').copy()
ids = ids[0]

In [14]:
df.idzona.fillna(df.ciudad.map(ids), inplace = True)
df.idzona.fillna(0, inplace=True)

In [15]:
df.drop(['lat', 'lng', 'titulo', 'descripcion', 'tipodepropiedad', 'direccion', 'ciudad', 'provincia', 'fecha'], axis=1, inplace=True)
df.columns

Index(['antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos',
       'metrostotales', 'idzona', 'gimnasio', 'usosmultiples', 'piscina',
       'escuelascercanas', 'centroscomercialescercanos', 'precio'],
      dtype='object')

In [16]:
features = df.drop('precio', axis=1).columns
X_train,X_test,y_train,y_test = train_test_split(df[features], df.precio,test_size=0.3, random_state = 0)

In [18]:
clf = RandomForestClassifier(n_estimators = 40, max_depth = 8)
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=8, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=40,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [19]:
for feature in zip(features, clf.feature_importances_):
    print(feature)

('antiguedad', 0.06981961025465627)
('habitaciones', 0.07563835410468026)
('garages', 0.05849917232537383)
('banos', 0.09964378199796485)
('metroscubiertos', 0.24873160160141014)
('metrostotales', 0.2001559753587055)
('idzona', 0.17027365700823022)
('gimnasio', 0.01772235947110984)
('usosmultiples', 0.013110981652734263)
('piscina', 0.016898544673469483)
('escuelascercanas', 0.014214032371879954)
('centroscomercialescercanos', 0.015291929179785394)


In [20]:
preds = clf.predict(X_test)

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, preds)

In [ ]:
x = df.drop(columns = ['id', 'titulo', 'descripcion', 'direccion', 'ciudad', 'lat', 'lng']).copy()

In [ ]:
x.metrostotales.fillna(value='0', inplace = True)
x.metroscubiertos.fillna(value='0', inplace = True)
x.metrostotales = x.metrostotales.astype(int)
x.metroscubiertos = x.metroscubiertos.astype(int)
x['metrostotales'] = x[['metrostotales', 'metroscubiertos']].max(axis=1)
x['metroscubiertos'] = x[['metrostotales', 'metroscubiertos']].min(axis=1)
x['patio'] = x['metrostotales'] - x['metroscubiertos']

In [ ]:
x['fecha'] = pd.to_datetime(x['fecha'])
x['anio'] = x.fecha.dt.year
x['mes'] = x.fecha.dt.month
x['dia'] = x.fecha.dt.day
x = x.drop(columns = 'fecha')

In [ ]:
transformador_numeros = SimpleImputer(fill_value = 0)
#transformador_catboost = Pipeline(steps=[('imputer1', SimpleImputer(strategy='most_frequent')),
#                                           ('catboost', ce.CatBoostEncoder(handle_unknown='ignore'))])
transformador_onehot = Pipeline(steps=[('imputer2', SimpleImputer(strategy='most_frequent')),
                                        ('onehot', OneHotEncoder(handle_unknown='ignore'))])
cols_numericas = ['antiguedad', 'habitaciones','garages', 'banos', 'metroscubiertos',
                  'metrostotales', 'idzona', 'gimnasio', 'usosmultiples',
                  'piscina', 'escuelascercanas','centroscomercialescercanos', 'patio', 'anio', 'mes', 'dia']
#cols_catboost = ['ciudad']
cols_onehot = ['tipodepropiedad', 'provincia']

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', transformador_numeros, cols_numericas),
#        ('cat', transformador_catboost, cols_catboost),
        ('one', transformador_onehot, cols_onehot)
    ])

In [ ]:
features = x.drop('precio', axis=1).columns
X_train,X_test,y_train,y_test = train_test_split(x[features], x.precio,test_size=0.3, random_state = 0)
X_train.columns

In [ ]:
clf = RandomForestClassifier(n_estimators=10000, random_state=0)
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', clf)
                             ])

In [ ]:
my_pipeline.fit(X_train,y_train)
#X_train.columns

In [ ]:
a = SimpleImputer()
x[cols_numericas] = a.fit_transform(x[cols_numericas])

In [ ]:
x.dtypes